<H1>Stiff Skin Syndrome, Loeys et al. (2010)</H1>
<p>This notebook creates GA4GH Phenopackets from the clinical data in <a href="https://pubmed.ncbi.nlm.nih.gov/20375004/" target="__blank">Loeys BL, et al., Mutations in fibrillin-1 cause congenital scleroderma: stiff skin syndrome. Sci Transl Med. 2010 Mar 17;2(23):23ra20. PMID:20375004</a>.</p>

In [1]:
import phenopackets as php
from google.protobuf.json_format import MessageToDict, MessageToJson
from google.protobuf.json_format import Parse, ParseDict
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
from pyphetools.creation import *
import importlib.metadata
__version__ = importlib.metadata.version("pyphetools")
print(f"Using pyphetools version {__version__}")

Using pyphetools version 0.4.13


In [2]:
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199")
metadata.default_versions_with_hpo(version=hpo_version)

In [3]:
df = pd.read_table('input/stiffskin_loeys2010.tsv')

In [4]:
df

,Identifier,1-II:1,1-III:2,2-III:1,2-IV:2,3-I:1,3-II:2,3-II:3,4-II:1
0,Age(yrs),51,25,70,37,54,22,21,54
1,Sex,M,M,M,F,F,F,F,M
2,Height(cm),173,165,163,155,145,160,159,169
3,Stiff skin,++,+++,+++,+++,+++,+++,+++,+
4,Surgery,"Appendectomy, Herniorraphy, Coronaryarterybypass",Cesarean section,Gastric ulcer surgery,Cesarean Sections,Hysterectomy,Achilles tendon lengthening,Achillestendonlengthening,.
5,Ectopia Lentis,─,─,─*,─,─,─,─,─
6,Cataract,─,─,+,─,+,─,─,─
7,Myopia,+,+,─,─,+,+,+,+
8,Arachnodactyly,─,─,─,─,─,─,─,─
9,Camptodactyly,++,++,++,++,++,++,++,++


In [5]:
# convert to row-based format
dft = df.transpose()
dft.columns = dft.iloc[0]
dft.drop(dft.index[0], inplace=True)
dft['patient_id'] = dft.index
dft.head()

Identifier,Age(yrs),Sex,Height(cm),Stiff skin,Surgery,Ectopia Lentis,Cataract,Myopia,Arachnodactyly,Camptodactyly,Elbowcontractures,Limitedshoulderelevation,Kneecontractures,MCP/IPnodules,Aorta,Neuropathy,GER,Variant,patient_id
1-II:1,51,M,173,++,"Appendectomy, Herniorraphy, Coronaryarterybypass",─,─,+,─,++,++,─,++,++,nl,─,+,c.4710G>C,1-II:1
1-III:2,25,M,165,+++,Cesarean section,─,─,+,─,++,++,++,++,++,nl,+,+,c.4710G>C,1-III:2
2-III:1,70,M,163,+++,Gastric ulcer surgery,─*,+,─,─,++,++,++,++,++,nl,─,+,c.4710G>T,2-III:1
2-IV:2,37,F,155,+++,Cesarean Sections,─,─,─,─,++,++,++,++,++,nl,─,+,c.4710G>T,2-IV:2
3-I:1,54,F,145,+++,Hysterectomy,─,+,+,─,++,++,++,++,+,nl,─,─,c.4691G>C,3-I:1


<h2>Create mappers to convert data</h2>

In [6]:
column_mapper_d = defaultdict(ColumnMapper)

In [7]:
symbol_d = {'+': 'Stiff skin', '++': 'Stiff skin', '+++': 'Stiff skin'}
stiffSkinMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=symbol_d)
stiffSkinMapper.preview_column(dft['Stiff skin'])
column_mapper_d['Stiff skin'] = stiffSkinMapper

In [8]:
items = {
  'Ectopia Lentis': ["Ectopia Lentis","HP:0001083"],
  'Cataract': ['Cataract', 'HP:0000518'],
  'Myopia': ["Myopia", "HP:0000545"],
  'Arachnodactyly': ['Arachnodactyly', 'HP:0001166'],
  'Camptodactyly': ['Camptodactyly', 'HP:0012385'],
  'Elbowcontractures': ['Elbow contracture', 'HP:0034391'],
  'Limitedshoulderelevation': ['Limited shoulder flexion','HP:0033482'],
  'Neuropathy': ["Entrapment neuropathy",'HP:0012181'],
  'GER': ['Gastroesophageal reflux', 'HP:0002020']
}

item_column_mapper_d = hpo_cr.initialize_simple_column_maps(column_name_to_hpo_label_map=items, 
                                                            observed={'+', '++'},
                                                              excluded='-')

# Transfer to column_mapper_d
for k, v in item_column_mapper_d.items():
    column_mapper_d[k] = v

In [9]:
# Aorta
# Bicuspid aortic valve HP:0001647
mapper = SimpleColumnMapper(hpo_id='HP:0001647',
    hpo_label='Bicuspid aortic valve',
    observed='BAV',
    excluded='─')
mapper.preview_column(dft['Aorta'])
column_mapper_d['Aorta'] = mapper

<h2>Demographic columns</h2>

In [10]:
ageMapper = AgeColumnMapper.by_year('Age(yrs)')
#ageMapper.preview_column(dft['Age(yrs)'])

In [11]:
sexMapper = SexColumnMapper(male_symbol='M', female_symbol='F', column_name='Sex')
#sexMapper.preview_column(dft['Sex'])

<h2>Variant data</H2>

In [12]:
genome = 'hg38'
transcript='NM_000138.5' # FBN1
varMapper = VariantColumnMapper(assembly=genome,
                                column_name='Variant', 
                                transcript=transcript, 
                               default_genotype='heterozygous')
#varMapper.preview_column(dft['Variant'])

In [13]:
pmid = "PMID:20375004"
encoder = CohortEncoder(df=dft, 
                        hpo_cr=hpo_cr, 
                        column_mapper_d=column_mapper_d, 
                        individual_column_name="patient_id", 
                        agemapper=ageMapper, 
                        sexmapper=sexMapper,
                        metadata=metadata,
                        variant_mapper=varMapper,
                        pmid=pmid)
encoder.set_disease(disease_id='OMIM:184900', label='Stiff skin syndrome')

In [14]:
# Uncomment to check results
#individuals = encoder.get_individuals()
#i1 = individuals[0]
#phenopacket1 = i1.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh())
#json_string = MessageToJson(phenopacket1)
#print(json_string)

In [15]:
output_directory = "phenopackets"
encoder.output_phenopackets(outdir=output_directory)

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.4710G>C/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.4710G>C/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.4710G>T/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.4710G>T/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.4691G>C/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.4691G>C/NM_000138.5?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000138.5%3Ac.4691G>C/NM_000138.5?content-type=application%2Fjson
https://rest.